In [1]:
%matplotlib notebook
import os
from adm_library import *

Scratch

In [2]:
results = read_encoder_coeffs_from_file()
print(results[0],'\n')
print(results[1],'\n')

                            X         Y         Z        Rx        Ry
Encoder Coefficient                                                  
C1                   1.000457  -0.00207 -0.000724         0         0
C2                   0.001471  1.001223 -0.001685         0         0
C3                  -0.001434 -0.016575  1.013391         0         0
C4                   0.023028 -0.019712  0.005225         0         0
C5                   0.000001  0.000036 -0.000034         0         0
C6                  -0.000003  0.000055 -0.000038         0         0
C7                   0.000139 -0.000463  0.000226         0         0
C8                        0.0      -0.0       0.0         0         0
C9                        0.0      -0.0       0.0         0         0
C10                 -0.000002  0.000008 -0.000004         0         0
C11                  0.000003 -0.000012  0.000004         0         0
C12                 -0.000005  0.000077 -0.000042         0         0
C13                 

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [3]:
fname = 'files/Cal 5DoF inputs 20230509.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     0.991107   0.115821   0.754433  0.021212   0.045726
PR2  -109.485707 -46.959858   23.47922 -4.915799  10.693732
PR3   111.329338 -46.602151  23.152164 -4.873486 -10.586882
PR4   103.162412  58.543344  31.771468  5.291407 -10.304568
PR5  -101.345145  58.305984  31.898431  5.265896  10.417492 

Provided 5DOF encoder values: 
                X.1        Y.1        Z.1      Rx.1       Ry.1
Name.1                                                       
PR1       0.963670   0.152189   0.809752  0.023102   0.046499
PR2    -109.458516 -46.889335  23.527226 -4.908509  10.696670
PR3     111.210359 -46.491038  23.281909 -4.879384 -10.582752
PR4     103.036944  58.601437  31.983290  5.287024 -10.308359
PR5    -101.335083  58.326220  32.034128  5.274356  10.412763


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [4]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [5]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [6]:
#p_null_PATB_baseline_encoder = pd.DataFrame(dict({'X':88.435, 'Y':109.266,'Z':0,
#                                                  'Rx':0.1018,'Ry':0.0856,
#                                                  'z_sMATF':4.027174*1000.,
#                                                  'z_PATB':3.455684*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':0.0041,
#                                                  'sMATF AC EL':0.006,
#                                                  'PATB AC AZ':0.0041,
#                                                  'PATB AC EL':0.0035,
#                                                  'sMATF Pri LED X':811.9,
#                                                  'sMATF Pri LED Y':604.6,
#                                                  'PATB Pri LED X':812.6,
#                                                  'PATB Pri LED Y':605.9,
#                                                  'date':'4/1/2023'
#                                                 }),index=['PATB'])
p_null_PATB_baseline_encoder = get_data_from_ADM_log('SES CP2','retro',index_name='PATB',print_details=True) 
p_null_PATB_baseline = calculate_5DOF_from_encoders(p_null_PATB_baseline_encoder)

p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
p_null_PATB_baseline_encoder


X                               88.435
Y                              109.266
Z                                    0
Rx                              0.1018
Ry                              0.0856
z_sMATF                       4027.174
z_PATB                        3455.684
z_type                           retro
sMATF AC AZ                     0.0041
sMATF AC EL                      0.006
PATB AC AZ                      0.0041
PATB AC EL                      0.0035
sMATF Pri LED X                  811.9
sMATF Pri LED Y                  604.6
PATB Pri LED X                   812.6
PATB Pri LED Y                   605.9
date               2023-04-01 00:00:00
Name: PATB, dtype: object
[88.435, 109.266, 0.0, 1.0, 7820.7492250000005, 11939.058756, 0.0, 691627.9577128751, 1304533.194033096, 0.0, 9662.93871, 0.0, 0.0, 0.1018, 0.0856, 1.0, 0.010363240000000001, 0.007327359999999999, 0.001054977832, 0.0006272220159999999, 0.008714079999999999]
                            X         Y         Z    

,X,Y,Z,Rx,Ry,z_sMATF,z_PATB,z_type,sMATF AC AZ,sMATF AC EL,PATB AC AZ,PATB AC EL,sMATF Pri LED X,sMATF Pri LED Y,PATB Pri LED X,PATB Pri LED Y,date
PATB,88.435,109.266,0,0.1018,0.0856,4027.174,3455.684,retro,0.0041,0.006,0.0041,0.0035,811.9,604.6,812.6,605.9,2023-04-01


### 1.1.2 Update the PATB pose to account for imperfect nulling.

In [7]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PATB_baseline_encoder DataFrame with the optimal ones.
optimize_PATB_for_imperfect_nulling = True

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PATB_baseline_encoder_original = p_null_PATB_baseline_encoder.copy()

if optimize_PATB_for_imperfect_nulling == True:
    p_null_PATB_baseline_encoder = optimize_p_null_PATB_encoders(p_null_PATB_baseline_encoder,p_null_offset)

Starting GSARX/RY:  0.1018 0.0856
Optimal GSARX/RY:  0.1032160175 0.08766031499999999
Starting HTSA/VTSA:  88.435 109.266
Optimal HTSA/VTSA:  88.446418 109.33648500000001


#### 1.1.5 Form a dataframe with the values supplied so far

In [8]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PATB_baseline_encoder.loc['PATB','z_sMATF'] - \
    p_null_PATB_baseline_encoder.loc['PATB','z_PATB']
df = pd.concat([df,p_null_PATB_baseline])
df.loc['PATB','color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,0.991107,0.115821,0.754433,0.021212,0.045726,crimson,1.031195,248.370010,1268.027256
PR2,-109.485707,-46.959858,23.47922,-4.915799,10.693732,crimson,239.765216,350.383100,1220.379520
PR3,111.329338,-46.602151,23.152164,-4.873486,-10.586882,crimson,-237.396982,349.604227,1221.065684
PR4,103.162412,58.543344,31.771468,5.291407,-10.304568,crimson,-231.135803,128.737997,1264.746858
PR5,-101.345145,58.305984,31.898431,5.265896,10.417492,crimson,233.640931,129.254531,1264.233772
sMask,1.469,115.184948,588.226533,NaN,NaN,g,NaN,NaN,NaN
PATB,88.761271,109.507097,-0.683007,0.101093,0.083517,darkblue,1.883448,246.601719,1268.371354
sMPA,2.022,248.477,1268.761,NaN,NaN,purple,0.000761,0.411546,0.911389


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [9]:
start_coords = np.array(df.loc['PATB',['X','Y','Z']])

rx = df.loc['PATB','Rx']
ry = df.loc['PATB','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PATB',['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,0.991107,0.115821,0.754433,0.021212,0.045726,crimson,1.031195,248.370010,1268.027256
PR2,-109.485707,-46.959858,23.47922,-4.915799,10.693732,crimson,239.765216,350.383100,1220.379520
PR3,111.329338,-46.602151,23.152164,-4.873486,-10.586882,crimson,-237.396982,349.604227,1221.065684
PR4,103.162412,58.543344,31.771468,5.291407,-10.304568,crimson,-231.135803,128.737997,1264.746858
PR5,-101.345145,58.305984,31.898431,5.265896,10.417492,crimson,233.640931,129.254531,1264.233772
sMask,1.469,115.184948,588.226533,NaN,NaN,g,NaN,NaN,NaN
PATB,88.761271,109.507097,-0.683007,0.101093,0.083517,darkblue,1.883448,246.601719,1268.371354
sMPA,2.022,248.477,1268.761,NaN,NaN,purple,0.000761,0.411546,0.911389
sMATF,-0.420184,108.71451,571.608564,0.101093,0.083517,orange,0.833026,109.068985,560.985449


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [10]:
#p_null_PATB_update_encoder = pd.DataFrame(dict({'X':88.526, 'Y':108.364,'Z':0,
#                                                  'Rx':0.1287,'Ry':0.0788,
#                                                  'z_sMATF':4.031636*1000.,
#                                                  'z_PATB':3.458517*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':-0.0022,
#                                                  'sMATF AC EL':0.0042,
#                                                  'PATB AC AZ':-0.0008,
#                                                  'PATB AC EL':0.0001,
#                                                  'sMATF Pri LED X':811.7,
#                                                  'sMATF Pri LED Y':597.6,
#                                                  'PATB Pri LED X':812.1,
#                                                  'PATB Pri LED Y':598.8,
#                                                  'date':'4/3/2023'
#                                                 }),index=['PATB_update'])
p_null_PATB_update_encoder = get_data_from_ADM_log('SES WP2','retro',index_name='PATB_update',print_details=True) 
p_null_PATB_update = calculate_5DOF_from_encoders(p_null_PATB_update_encoder)
p_null_PATB_update

X                               88.526
Y                              108.364
Z                                    0
Rx                              0.1287
Ry                              0.0788
z_sMATF                       4031.636
z_PATB                        3458.517
z_type                           retro
sMATF AC AZ                    -0.0022
sMATF AC EL                     0.0042
PATB AC AZ                     -0.0008
PATB AC EL                      0.0001
sMATF Pri LED X                  811.7
sMATF Pri LED Y                  597.6
PATB Pri LED X                   812.1
PATB Pri LED Y                   598.8
date               2023-04-03 00:00:00
Name: PATB_update, dtype: object
[88.526, 108.364, 0.0, 1.0, 7836.8526759999995, 11742.756496000002, 0.0, 693765.219995576, 1272492.0649325443, 0.0, 9593.031464, 0.0, 0.0, 0.1287, 0.0788, 1.0, 0.016563690000000002, 0.006209439999999999, 0.0021317469030000006, 0.0004893038719999999, 0.01014156]
                            X         Y   

,X,Y,Z,Rx,Ry
PATB_update,88.850425,108.603476,-0.679239,0.127999,0.076715


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [11]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PATB_baseline_encoder DataFrame with the optimal ones.
optimize_PATB_for_imperfect_nulling = True

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PATB_update_encoder_original = p_null_PATB_update_encoder.copy()

if optimize_PATB_for_imperfect_nulling == True:
    p_null_PATB_update_encoder = optimize_p_null_PATB_encoders(p_null_PATB_update_encoder,p_null_offset)

Starting GSARX/RY:  0.1287 0.0788
Optimal GSARX/RY:  0.1298684923 0.0829718906
Starting HTSA/VTSA:  88.526 108.364
Optimal HTSA/VTSA:  88.54602399999999 108.421932


#### 1.2.2 Use the newly-provided data to calculate an updated position for sMATF

In [12]:
dz_update = p_null_PATB_update_encoder.loc['PATB_update','z_sMATF'] - \
    p_null_PATB_update_encoder.loc['PATB_update','z_PATB']
df = pd.concat([df,p_null_PATB_update])
df.loc['PATB_update','color'] = 'purple'
update_uvec(df,'PATB_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PATB_update',['X','Y','Z']])

rx = df.loc['PATB_update','Rx']
ry = df.loc['PATB_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PATB_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,0.991107,0.115821,0.754433,0.021212,0.045726,crimson,1.031195,248.370010,1268.027256
PR2,-109.485707,-46.959858,23.47922,-4.915799,10.693732,crimson,239.765216,350.383100,1220.379520
PR3,111.329338,-46.602151,23.152164,-4.873486,-10.586882,crimson,-237.396982,349.604227,1221.065684
PR4,103.162412,58.543344,31.771468,5.291407,-10.304568,crimson,-231.135803,128.737997,1264.746858
PR5,-101.345145,58.305984,31.898431,5.265896,10.417492,crimson,233.640931,129.254531,1264.233772
sMask,1.469,115.184948,588.226533,NaN,NaN,g,NaN,NaN,NaN
PATB,88.761271,109.507097,-0.683007,0.101093,0.083517,darkblue,1.883448,246.601719,1268.371354
sMPA,2.022,248.477,1268.761,NaN,NaN,purple,0.000761,0.411546,0.911389
sMATF,-0.420184,108.71451,571.608564,0.101093,0.083517,orange,0.833026,109.068985,560.985449
PATB_update,88.850425,108.603476,-0.679239,0.127999,0.076715,purple,1.730051,246.006097,1268.487231


In [13]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PATB->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  1.63279206946072  mm


## 1.3 Calculate deltas between baseline & updated ADM measurements

In [14]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PATB_update',['X','Y','Z','Rx','Ry']] - df.loc['PATB',['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PATB (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')


sMATF (X,Y,Z) deltas: 
 X    0.024668
Y   -0.863524
Z     1.59586
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X     0.089155
Y    -0.903621
Z     0.003768
Rx    0.026906
Ry   -0.006802
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [15]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X     -0.395516
Y    107.850987
Z    573.204423
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [16]:
update_rot1 = R.from_euler('XY',[df.loc['PATB','Rx']+GSA_angle_WCS_deg,df.loc['PATB','Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [17]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,1.084762,-0.479583,2.299526,0.048118,0.038924,crimson,0.877799,247.774531,1268.143863
PR2,-109.393668,-47.568351,24.989313,-4.888773,10.686960,crimson,239.615135,349.815221,1220.571893
PR3,111.421405,-47.205647,24.688189,-4.846697,-10.593655,crimson,-237.547122,349.025561,1221.202018
PR4,103.251162,57.935609,33.355916,5.318418,-10.311345,crimson,-231.286184,128.138980,1264.780198
PR5,-101.256403,57.693704,33.458906,5.292697,10.410714,crimson,233.490595,128.665947,1264.321582
sMask,1.491587,114.313661,589.82565,NaN,NaN,g,NaN,NaN,NaN
PATB,88.852687,108.914281,0.923697,0.127999,0.076715,darkblue,1.730051,246.006097,1268.487231
sMPA,1.962261,247.286127,1270.422696,NaN,NaN,purple,0.000646,0.411118,0.911582
sMATF,-0.395516,107.850987,573.204423,0.127999,0.076715,orange,0.765181,108.805548,561.036700
PATB_update,88.941861,108.01066,0.927051,0.154906,0.069913,purple,1.576654,245.410417,1268.602810


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [18]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

ValueError: operands could not be broadcast together with shapes (21,) (11,) 

#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [ ]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

# 2 Write the new poses to an Excel file in the current directory

In [ ]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from ADM update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,p_null_PATB_baseline_encoder_original=p_null_PATB_baseline_encoder_original,
                            p_null_PATB_update_encoder_original=p_null_PATB_update_encoder_original)